### Обучение с подкреплением


В некоторых случаях существующие методы обучения без учителя нам не подходят. В то же время у нас нет возможности создать качественную обучающую выборку. При этом мы можем постфактум оценить действия нашей модели и, используя эту оценку, подстроить модель так, чтобы она чаще совершала желательные действия и реже &mdash; нежелательные. В литературе такую оценку называют вознаграждением (reward). В этом случае обучение строится таким образом, чтобы модель стремилась максимизировать получаемое вознаграждение.

<center><img src="https://edunet.kea.su/repo/EduNet-content/L15/out/branches_of_machine_learning.png" alt="Drawing" width="350"></center>


<center><em> Branches of Machine Learning </em></center>

## Терминология: агент, функция награды, состояние среды


Агент и среда — ключевые понятия в обучении с подкреплением.

**Агент** — программа, принимающая решение о дальнейших действиях на основе информации о состоянии среды.

**Среда** — это мир, в котором агент должен "выживать", т.е. всё, с чем агент может прямо или косвенно взаимодействовать. Среда обладает состоянием (State), агент может влиять на среду, совершая какие-то действия (Actions), переводя среду при этом из одного состояния в другое и получая какое-то вознаграждение. Среда описывается пространством возможных состояний. Конкретное состояние — вектор в этом пространстве.

В зависимости от конкретной задачи, агент может наблюдать либо полное состояние среды, либо только некоторую его часть. Во втором случае агенту может потребоваться какое-то внутреннее представление полного состояния, которое будет обновляться по мере получения новых данных.

**Функция награды** — вводимая программистом формула вычисления ценности действия на основе финального результата, ожидания этого результата, промежуточных результатов и любых других параметров, которые будут подсказывать путь к наилучшей последовательности действий агента. Это некоторый аналог функции потерь, без которой непонятно, чему учиться. Например, в шахматах истинная награда — это победа, но взятая фигура соперника тоже ценна и должна увеличивать награду, если мы хотим подсказать агенту, что брать чужие фигуры полезно. Может ли после этого агент получать мат, позарившись на незащищенную фигуру? Да, ровно как и неопытный шахматный игрок.

Попытка передать через дополнительные неосновные награды подсказки к получению основной награды называется **reward shaping**.

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L15/rl_scheme.png" width="400">


- Неопределенность: переходы и награды случайны
- Отложенность награды может не зависеть от действия

## Классические примеры задач RL


* Обучение автопилота вождению машины
* Управление роботами, БПЛ
* Обучение нейронной сети игре в игры/видеоигры
* Составление расписаний с помощью нейросети
* Рекомендательные системы
* Трейдинг, принятие решений в условиях неопределенности, управление инвестициями
* Drug discovery: есть много свойств химических веществ, которые подсчитываются специальными программами. Выходы, которые, очевидно, недиффиренцируемы. Если мы хотим генерировать вещества с заданными свойствами, то один из подходов — RL
* Не так давно одна российская металлургическая компания использовала RL для оптимизации работы прокатного станка
* NAS (Neural Architecture Search): при поиске оптимальных топологий сети можно решать задачу с помощью RL, награждая нашего агента за нахождение хороших топологий и наказывая за плохие


Во всех этих задачах невозможно или непрактично собирать и размечать датасет, поэтому используется обучение с подкреплением.



Сделаем также важное предположение о природе нашего процесса: **он полностью описывается своим текущим состоянием**. Все, что произойдет в будущем, не зависит от информации из прошлого, кроме той, что мы уже наблюдаем в настоящем. Такой процесс называется __марковским__.


Приведем классические примеры марковского процесса:

1. Игральный кубик. Мы знаем, что на нем выпадет любая из граней с некоторой фиксированной вероятностью. На это никак не влияет то, что до этого на кубике выпало 6 шестерок подряд. Это может повлиять на нашу оценку вероятностей выпадения той или иной грани, но не на реальную вероятность.

2. Шахматы. "Текущая позиция на доске + чей ход" однозначно описывает игру.

3. А подходит ли покер?

С одной стороны, да. Текущее количество денег у каждого из игроков однозначно описывает игру. Но это если мы не учитываем блеф и другие факторы, которые могут сработать / не сработать в зависимости от предыдущих ситуаций в игре. То есть, все зависит от того, как именно мы будем описывать покер и какие допущения накладываем.

И получение хорошего представления S — тоже важная задача.

### Определение Markov Property

"*The future is independent of the past given the present*"

Состояние $S_{t}$ является Марковым тогда и только тогда, когда:
$$ \large
p\left(r_{t}, s_{t+1} \mid s_{0}, a_{0}, r_{0}, \ldots, s_{t}, a_{t}\right)=p\left(r_{t}, s_{t+1} \mid s_{t}, a_{t}\right)
$$

**Что происходит потом, зависит только от предыдущего состояния**
- Вся актуальная информация берется из истории.
- Как только состояние среды станет известно, историю можно выбросить.
- То есть, состояние среды — достаточная статистика для будущего обучения.

Вероятность перехода в новое состояние зависит только от текущего состояния и текущего действия.

Марковский процесс описывает **полностью наблюдаемые (fully observable)** среды. Можно описать их так:

* Информации, которую получает агент в момент времени t, достаточно, чтобы принять оптимальное решение для момента времени t.
* Текущее состояние среды содержит всю релевантную информацию из прошлого.
* Текущее состояние — это достаточная статистика будущего $\rightarrow$ не существует никакой дополнительной информации, которая могла бы улучшить наше описание будущего.

## Markov process

Представим себе, что студент живет вот по такой схеме. Заметим, что влиять в такой схеме на свои решения он не может: все решается подкидыванием кубика.

<img src="https://edunet.kea.su/repo/EduNet-content/L15/out/markov_process.png" width="900">

### Определение

Марковский процесс (цепь) — это кортеж $(S, P)$, где
- $S$ — принимает дискретные (конечные) значения,
- $P$ — матрица переходов (transition matrix):
$$\large
P_{s s^{\prime}}=\operatorname{Pr}\left(S_{t+1}=s^{\prime} \mid S_{t}=s\right)
$$

Строго говоря, необходимо еще распределение начальных состояний (но мы предполагаем, что оно вырождено, т.е. мы знаем, где начинаем, с вероятностью 1).

Марковский процесс — основа для RL. Мы будем постепенно усложнять эту модель, добавляя rewards и actions.

В определении нам встретилась матрица переходов, зададим ее тоже формально.

## Матрица состояний


### Transition matrix

Пусть $S_{t}$ — последовательность дискретных состояний.

Поскольку последовательность задается распределением $\operatorname{Pr}\left(S_{t+1} \mid S_{t}\right)$, естественно упорядочить его в матрицу $ P_{s s^{\prime}}=\operatorname{Pr}\left(S_{t+1}=s^{\prime} \mid S_{t}=s\right)$

$$
\mathcal{P}=\text { from }\left[\begin{array}{ccc}
\mathcal{P}_{11} & \ldots & \mathcal{P}_{1 n} \\
\vdots & & \\
\mathcal{P}_{n 1} & \ldots & \mathcal{P}_{n n}
\end{array}\right]
$$


## Награда (Reward)

Наши состояния, очевидно, неравноценны. Давайте к каждому состоянию привяжем награду $R_s$.
Она может быть как нулевой, так положительной либо отрицательной.

Например, в марковском процессе принятия решений (MDP), описывающем игру в шахматы или GO, награда агента может быть положительной только при переходе в состояние "партия выиграна"

На всех остальных ходах награда будет нулевой.

Для перемещающегося робота награда на каждом шаге может быть отрицательной, так как на перемещение требуется энергия и т.п.

Для покера наградой можно назначить изменение текущей суммы игрока после сыгранной раздачи.

Схема для студента в этом случае модифицируется следующим образом. Заметьте, награды тут расставляются с точки зрения актора — студента:



<img src="https://edunet.kea.su/repo/EduNet-content/L15/out/markov_reward.png" width="500">

## Суммарная награда (Return)

Для турнира по шахматам, где играется 10 партий, финальный reward будет складываться из reward за каждую партию в отдельности.

Если у нас есть тест из 40 вопросов, то опять же, нам не важно, как именно мы получили 30 баллов, главное — мы их получили.

То есть, в таких случаях мы можем считать Return следующим образом:

$$Return = \sum_i {R_i}$$


А теперь представим себе, что мы можем работать 2 года бесплатно и получить разом 5 миллионов рублей. А можем работать 2 года и каждый год получать 70 тысяч.


Или мы можем 6 лет учиться, получая сторонними активностями 20 тысяч в месяц, а затем сразу начать получать 150 тысяч в месяц, а затем и больше. А можем сразу пойти на работу и начать зарабатывать 50 тысяч в месяц, постепенно дойдя до 100.

Что лучше: опубликовать две статьи в журнале с импакт фактором 5 в этом году или 1, но через 2 года и в журнале с импакт-фактором 20?

Что выгоднее?

Очевидно, в таких ситуациях лучше учитывать не только Reward, но и то, когда он получен.

### Дисконтирование (discounting)

Поэтому при оценке кумулятивной награды на шаге $t$ $(G_t)$ используется коэффициент дисконтирования $\gamma$, который показывает, насколько ценными являются будущие награды на текущий момент (см. [Markov Decision Processes (David Silver Lectures)](https://www.davidsilver.uk/wp-content/uploads/2020/03/MDP.pdf)):

$$\large G_t = R_{t+1} + \gamma R_{t+2} + ... =\sum^{\infty}_{k=0} \gamma ^ kR_{t+k+1},
$$

где:
- $R_{t+1}, R_{t+1}, \dots$ — вознаграждения по рёбрам переходов,
- дисконтирование $\gamma \in[0,1]$ — это текущая стоимость будущих вознаграждений,
- $\gamma^{k} R$ — ценность получения награды $R$ после $k+1$ шагов.

При этом:
- немедленное вознаграждение ценится выше отложенного вознаграждения,
- $\gamma$ близко к 0 приводит к «близорукости»,
- $\gamma$ близко к 1 приводит к «дальновидной» оценке.


Какое значение $\gamma$ выбрать?


**Можно ли его выбрать равным 1?**

Да, мы уже это делали ранее.

**Можно ли его выбрать равным 0?**

 Тоже да. В этом случае у нас получится "жадный" алгоритм: мы всегда выбираем решение, которое дает максимальную награду сейчас, нас не волнуют будущие награды.

**Может $\gamma = 0$ привести к проблемам?**

Да, например:

* Вариант 1: Получить сегодня 1000 рублей.

* Вариант 2: Получить завтра 100000 рублей

Обычно, второй вариант более предпочтителен. Но жадный алгоритм его не увидит.





**Может $\gamma = 1$ привести к проблемам?**

Представим себе генератор шуток про медведя. Большая часть людей знает эту шутку, поэтому было бы хорошо посередине вставлять дополнительные детали &mdash; "для ценителей". Но заканчиваться шутка должна одинаково.

Мы решили написать программу, которая генерирует шутки про медведя. Шутка должна начинаться с "Шел медведь по лесу" и заканчиваться "Сел в машину и сгорел". Как бы это описать?

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L15/cat_comedian.png" alt="Drawing" width="500"/>

Есть состояния, а модель сама решает, куда ей переходить. Вероятностей мы тут не добавили. Что модель решила, то и будет.

Сразу видим и проблему, которую мы получаем при $\gamma = 1$, пусть и в утрированном виде. Ничто не запрещает нашей модели бесконечно добавлять детали к анекдоту, тем самым увеличивая $G_t$.
Работать с такой моделью невозможно.

Если в какой-то момент обучения наша модель стала походить на эту, то ничего осмысленного мы и далее не получим. А вот если введением дискаунта дать модели понять, что слушатели анекдота все же хотят вскоре услышать его конец, то проблема уйдет.

**Обычно $0 \le \gamma \le 1$**

Обычно $\gamma$ ставят равной чему-то между двумя этими крайностями.

Близость $\gamma$ к 0 отражает нашу "нетерпеливость" — насколько важно получить награду именно сейчас.

Наличие такого $\gamma$ позволяет нам не делать различия между моделями с ограниченным числом шагов и неограниченным: теперь в любом случае return будет конечным числом, т.к. выражение для return ограничено сверху суммой бесконечно убывающей геометрической прогрессии.

Пусть $R_{max} = max R_i$ — максимальная награда, которую мы в принципе можем получить в каком-то состоянии.

$\displaystyle G_t = R_{t+1} + \gamma \cdot R_{t+2} + \gamma^2 \cdot R_{t+3} + ... \le R_{max} + \gamma \cdot R_{max} + \gamma^2 \cdot R_{max} + ... = R_{max} (1 + \gamma + \gamma^2 + ... ) \le R_{max} \cdot \frac{1}{1-\gamma} = const $

**Discounting makes sums finite**

<img src="https://edunet.kea.su/repo/EduNet-content/L15/out/discounting_makes_sums_finite.png" alt="Drawing" width="800"/>

\begin{equation}
G_{0}=\sum_{k=0}^{\infty} \gamma^{k}=\frac{1}{1-\gamma}
\end{equation}

Важно понимать, что выбор $\gamma$ меняет задачу, которую мы решаем, и, соответственно, меняет решение.

### Markov reward process (MRP)

MRP — это кортеж $(S, R, P, \gamma)$, где:

$S$ — дискретное состояние, принимает дискретные (конечные значения),

$R$ — функция rewards, $R_s = \mathbb{E}[R_{t+1}|S_t=s]$,

$P$ — матрица переходов (transition matrix):

$P_{ss^`}=Pr(S_{t+1}=s^`|S_t=s)$,

$\gamma$ — коэффициент дисконтирования (discount factor).

Осталось добавить actions.


# Марковский процесс принятия решений
Markov decision process (MDP)

До этого у нас получалось малость неадекватное представление процесса &mdash; студент не мог ни на что повлиять. На самом же деле студент может решать, куда ему надо: в аудиторию, домой спать или в бар. Однако наше действие не всегда определяет состояние, в которое мы перейдем.

Хочет наш студент пойти на лекцию, но по пути встречает товарища, и идут они в бар. Хочет он пойти спать, а по пути к выходу из университета встречает лектора с хорошей памятью и идет на лекцию.
И т.д.

Как это отразить на схеме так, чтобы это можно было прочитать? Введем промежуточные состояния, куда нас переводят действия студента. А уже из этих промежуточных состояний случайно будем переходить в состояния среды.



<img src="https://edunet.kea.su/repo/EduNet-content/L15/out/markov_decision_process_return_random.png" alt="Drawing" width="650"/>

## Формальное описание MDP

MDP — это кортеж $(S, A, R, P, \gamma)$, где:
- $S$ — состояния (дискретное пространство),
- $A$ — действия (дискретное пространство),
- $R$ — функция rewards: $R_{s}^{a}=\mathbb{E}\left[R_{t+1} \mid S_{t}=s, A_{t}=a\right]$,
- $P$ — матрица переходов (transition matrix): $P_{s s^{\prime}}^{a}=\operatorname{Pr}\left(S_{t+1}=s^{\prime} \mid S_{t}=s, A_{t}=a\right)$,
- $\gamma$ — discount factor.

### Пример

<img src="https://edunet.kea.su/repo/EduNet-content/L15/out/markov_decision_process_example.png" alt="Drawing" width="500"/>

- черные беззнаковые числа — вероятности переходов;
- знаковые — это награды.


Приведем другие классические примеры марковского процесса:

1. Игральный кубик. Мы знаем, что на нем выпадет любая из граней с некоторой фиксированной вероятностью. На это никак не влияет то, что до этого на кубике выпало 6 шестерок подряд. Это может повлиять на нашу оценку вероятностей выпадения той или иной грани, но не на реальную вероятность.

2. Шахматы. "Текущая позиция на доске + чей ход" однозначно описывает игру.

3. А подходит ли покер?

С одной стороны, да. Текущее количество денег у каждого из игроков однозначно описывает игру. Но это если мы не учитываем блеф и другие факторы, которые могут сработать / не сработать в зависимости от предыдущих ситуаций в игре. То есть, все зависит от того, как именно мы будем описывать покер и какие допущения накладываем.

И получение хорошего представления S — тоже важная задача.

## Нахождение лучшей последовательности переходов

Нужно найти последовательность переходов, которой будет соответствовать максимальная награда ($G_t$).

### Определение Политики

Политика (policy) $\pi$ — это функция, которая для текущего состояния $s$ дает распределение вероятностей на множестве действий $A$.
$$ \large
\pi(a|s)=\mathbb{P}\left[A_{t}=a \mid S_{t}=s\right]
$$
- Политика полностью определяет поведение агента
- Политики MDP зависят от текущего состояния среды (а не от прошлых состояний)
- т. е. политики являются стационарными (не зависящими от времени),
$$ \large
A_{t} \sim \pi\left(\cdot \mid S_{t}\right), \forall t>0
$$

Для нашего примера со студентом:


<img src="https://edunet.kea.su/repo/EduNet-content/L15/out/markov_policy_example.png" alt="Drawing" width="700"/>

### Value function

Для оценки политики вводятся функции $V$ (_state-value function_) и $Q$ (_action-value function_):

* $v_\pi(s)$ — измеряет ценность каждого состояния, а именно какое ожидаемое вознаграждение можно получить, если начать двигаться из состояния $s$ в течение всего оставшегося времени, придерживаясь политики $\pi$.

Так, например, агент может следовать политике, заключающейся в переходе между состояниями с высокой ценностью. Однако функция ценности состояния удалена от принятия решений агентом, поскольку агент зачастую не может сам выбрать следующее состояние, в которое перейдет, поскольку на него будет оказываться влияние случайности (например, броски кубиков) или внешних сил (ход оппонента). Поэтому при выборе оптимальной политики рассматривают ценность не состояний, а действий.

* $q_\pi(s,\color{red}a)$ — аналогично функции ценности состояния, но при условии, что в состоянии $s (s_0 = s)$ было выбрано действие $a$ (мы фиксируем действие). То есть в $q$ мы, в отличие от $v$, фиксируем наше первое действие (не обязательно соответствующее политике), а в $v$ мы первое действие выбираем согласно политике $\pi$.

Матожидание здесь берется по нашей политике, т.к. она стохастична.

### Определение Value Function

**State-value function** $v_{\pi}(s)$ MDP — ожидаемая выгода, начиная с состояния $s$, при условии, что агент следует политике $\pi$:
$$ \large
v_{\pi}(s)=\mathbb{E}_{\pi}\left[G_{t} \mid S_{t}=s\right] = \mathbb{E}_{\pi}\left[\sum_{k=0}\gamma^{k+1}R_{t+k+1} \mid S_{t}=s\right]
$$
**Action-value function** $q_{\pi}(s, a)$ — ожидаемая выгода, начиная с состояния $s$, действия $a$, и при условии следования политике $\pi$:
$$ \large
q_{\pi}(s,\color{red}a)=\mathbb{E}_{\pi}\left[G_{t} \mid S_{t}=s, \color{red}{A_{t}=a}\right] = \mathbb{E}_{\pi}\left[\sum_{k=0}\gamma^{k+1}R_{t+k+1} \mid S_{t}=s, \color{red}{A_{t}=a}\right]
$$

# Gym



Давайте разберемся, что такое Gymnasium, и какие возможности он предоставляет для обучения с подкреплением.

**Gymnasium** — это набор инструментов для разработки и сравнения алгоритмов RL, позволяющий стандартизировать взаимодействие между разными алгоритмами RL и средами. Также он предоставляет набор стандартных сред, которые могут, в том числе, использоваться для бенчмаркинга.

[Gymnasium projects](https://openai.com/projects/)

[Документация](https://gymnasium.farama.org/)

[Colab demonstration](https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb)

Для начала рассмотрим устройство среды Gym в целом.
__Среда__ — это некоторая модель мира, отвечающая за предоставление наблюдений и вознаграждений, в которой существует __агент__, Состояние среды будет изменяться в зависимости от действий агента.

Рассмотрим для начала стандартную среду MountainCar, в которой стоит задача — довести машину до вершины горы.

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L15/mountain-car-v0.gif"/>

In [ ]:
!pip install -q gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym

env = gym.make("MountainCar-v0")

В Gym среды представлены классом `gym.Env`, который является унифицированным интерфейсом среды со следующими атрибутами и методами:
* `action_space`: описание действий, допустимых в данной среде;
* `observation_space`: структура и допустимые значения наблюдений состояния среды;
* `reset()`: сбрасывает среду и возвращает случайное исходное состояние;
* `step(action)`: метод, продвигающий развитие окружающей среды на одно действие и возвращающий информацию о результате этого действия, а именно:
    1. observation &mdash; следующее наблюдение;
    2. reward &mdash; локальное вознаграждение;
    3. done &mdash; флаг конца эпизода.

Также в классе `gym.Env` есть несколько вспомогательных методов, например `render()`, позволяющий представить наблюдение в понятной человеку форме, но мы не будем их касаться в нашей лекции.

### Пространства действий и наблюдений

Действия агента могут быть дискретными, непрерывными и комбинированными. Дискретные действия представлены фиксированным набором и взаимно исключают друг друга, например нажатие/отпускание клавиши. Непрерывным действиям соответствуют значения в некотором диапазоне, например, поворот руля от -720 до 720 градусов. В среде мы не ограничены каким-то одним действием и вполне допустимо одновременное нажатие нескольких кнопок и одновременный поворот руля. Аналогичным образом наблюдения могут быть дискретыми (лампочка включена/выключена) или непрерывными (тензоры, соответствующие цветным изображениям).

Давайте посмотрим на то, как выглядят пространства действий и наблюдений в среде MountainCar:

In [ ]:
from warnings import simplefilter

simplefilter("ignore", category=DeprecationWarning)


# Action and observation space
action_space = env.action_space
obs_space = env.observation_space

print(f"The action space: {action_space}")
print(f"The observation space: {obs_space}")

The action space: Discrete(3)
The observation space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)


Мы видим, что пространство наблюдений и действий представлено некоторыми классами `Box` и `Discrete` соответственно. Что же это за классы? Для объединения нескольких пространств действий (например, непрерывных и дискретных) в одно действие, в Gym существует специальный класс контейнеров.

* Класс `Discrete` представляет набор $n$ взаимоисключающих элементов. Например, `Discrete(n=4)` может быть использован для пространства действий с 4 направлениями движения ($\leftarrow \downarrow \rightarrow \uparrow$)
* Класс `Box` представляет n-мерный тензор рациональных чисел в некотором диапазоне `[low, high]`. Например, нажатие педали газа от минимального значения 0 до максимального &mdash; 1, можно закодировать как `с`, аналогично наблюдение экрана игры, можно закодировать как `Box(low=0, high=255, shape=(100, 50, 3), dtype=np.float32)`

Давайте посмотрим на диапазон допустимых значений пространства наблюдений в среде MountainCar:

In [ ]:
print("Upper Bound for Env Observation", env.observation_space.high)
print("Lower Bound for Env Observation", env.observation_space.low)

Upper Bound for Env Observation [0.6  0.07]
Lower Bound for Env Observation [-1.2  -0.07]


Обе структуры данных происходят от класса `gym.Space`

In [ ]:
print(type(env.action_space))
print(type(env.observation_space))

<class 'gymnasium.spaces.discrete.Discrete'>
<class 'gymnasium.spaces.box.Box'>


Также стоит упомянуть еще один дочерний класс  `gym.Space` &mdash; `Tuple`, позволяющий объединять несколько экземпляров класса `gym.Space` вместе. Благодаря этому классу мы можем создавать пространства действий и наблюдений любой сложности.

В классе `gym.Space` реализованы методы, позволяющие взаимодействовать с пространствами действий и наблюдений:
* `sample()`: возвращает случайный пример из пространства наблюдений
* `contains(x)`: проверяет принадлежность аргумента к пространству наблюдений

Давайте возьмем случайное действие, доступное нам в исходном состоянии среды MountainCar:

In [ ]:
random_action = env.action_space.sample()  # random number from 0 to 2
print(random_action)

1


### Взаимодействие со средой

Давайте попробуем совершить случайное действие, которое мы выбрали выше, для этого перезапустим среду, чтобы вернуться в изначальное состояние, и сделаем шаг с помощью метода `step()`:

In [ ]:
# reset the environment and see the initial observation
obs, info = env.reset()
print(f"The initial observation is {obs}")

# Take the action and get the new observation space
new_obs, reward, done, truncated, info = env.step(random_action)
print(f"The new observation is {new_obs}")

The initial observation is [-0.48531637  0.        ]
The new observation is [-4.8560286e-01 -2.8648737e-04]


truncated (bool) — выполняется ли условие усечения вне области действия MDP. Как правило, это ограничение по времени, но его также можно использовать для обозначения того, что агент физически выходит за границы. Может использоваться для преждевременного завершения эпизода до того, как будет достигнуто терминальное состояние. Если это правда, пользователю необходимо вызвать reset

In [ ]:
truncated

False